# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [12]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [13]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [14]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [35]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


In [32]:
float(p[0].text)

15600.0

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [16]:
document = ET.parse( './data/mondial_database.xml' )

In [17]:
root = document.getroot()

In [18]:
root.tag

'mondial'

## 10 countries with the lowest infant mortality

In [19]:
countries = []
inf_mort_rate = []

In [20]:
for c in root.iter('country'):
    countries.append(c.findtext('name'))
    inf_mort_rate.append(c.findtext('infant_mortality'))

In [21]:
pd_inf_mort_rate = pd.DataFrame({'Infant mortality rate': inf_mort_rate}, index = countries)
pd_inf_mort_rate=pd_inf_mort_rate['Infant mortality rate'].astype('float')
pd_inf_mort_rate = pd_inf_mort_rate.dropna()

The answer is:

In [22]:
pd_inf_mort_rate.order().head(10)

Monaco            1.81
Japan             2.13
Bermuda           2.48
Norway            2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
Name: Infant mortality rate, dtype: float64

## 10 cities with the largest popullation

In [140]:
cities = []
population = []
country = []

In [142]:
# print names of all countries and their cities
for element in root.iterfind('country'):
    for subelement in element.getiterator('city'):
        for subsubelement in subelement.findall('population'):
            cities.append(subelement.find('name').text)
            population.append(subsubelement.text)
            country.append(element.find('name').text)
    
    for subelement in element.getiterator('province'):
        for subsubelement in subelement.findall('city'):
            for subsubsubelement in subsubelement.findall('population'):
                cities.append(subsubelement.find('name').text)
                population.append(subsubsubelement.text)
                country.append(element.find('name').text)

In [143]:
df_city_population = pd.DataFrame({'Country': country,'City': cities, 'Population': population})

In [144]:
df_city_population = df_city_population.drop_duplicates()
df_city_population['Population']=df_city_population['Population'].astype('float')
df_city_population = df_city_population.dropna()

In [145]:
df_city_population = df_city_population.groupby(['City']).max()

In [146]:
result = df_city_population.sort(['Population'], ascending=[0])

The answer is:

In [148]:
result.head(10)

,Country,Population
City,,
Shanghai,China,22315474
Istanbul,Turkey,13710512
Delhi,India,12877470
Mumbai,India,12442373
Moskva,Russia,11979529
Beijing,China,11716620
São Paulo,Brazil,11152344
Tianjin,China,11090314
Guangzhou,China,11071424


##10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [184]:
def latest_population(element):
    year_ref = 0
    for subelement in element.findall('population'):
        year = subelement.get('year')
        if year > year_ref:
            population= float(subelement.text)
            year_ref = year
    return population

In [188]:
population = []
country = []
ethnic_percentage = []
ethnic_name1 = []

In [190]:
for element in root.iterfind('country'):
    l = element
    for subelement in element.findall('ethnicgroup'):
        country.append(element.find('name').text)
        population.append(latest_population(element))
        ethnic_percentage.append(float(subelement.get('percentage')))
        ethnic_name.append(subelement.text)

In [191]:
df_ethtic_group = pd.DataFrame({'Country': country, 'Population': population, 
                                'Ethnic_percentage': ethnic_percentage, 'Ethnic_name': ethnic_name})
df_ethtic_group['Ethnic_percentage'] = df_ethtic_group['Ethnic_percentage'].astype('float')

In [192]:
df_ethtic_group['Ethtic_population'] = (df_ethtic_group['Ethnic_percentage']/100)*df_ethtic_group['Population']

In [194]:
df = df_ethtic_group[['Ethnic_name','Ethtic_population']]

In [197]:
result = df.groupby(['Ethnic_name']).sum().sort(['Ethtic_population'], ascending=[0])

The answer is:

In [198]:
result.head(10)

,Ethtic_population
Ethnic_name,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


##4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [266]:
def wrap_question4(root, tagname_prop, tagname_size):
    Prop = [] #River, Lake or Airport
    Size = [] #Length, Area or some such
    Country = []
    
    # Parcing XML here
    for element in root.iterfind(tagname_prop):
        Prop.append(element.find('name').text)
        Country.append(element.get('country'))
        if element.find(tagname_size) is not None:
            Size.append(element.find(tagname_size).text)
        else:
            Size.append(np.nan)
     
    df = pd.DataFrame({'Property': Prop, 'Size': Size, 'Country': Country})
    df['Size'] = df['Size'].astype('float')
    df_result = df.sort(['Size'], ascending=0).head(1).reset_index(drop = True)
    
    Property_result = df_result['Property'][0]
    Countries_result = []
    Countries_result_id = df_result['Country'][0].split(' ')
    
    # Getting proper names of the countries
    for element in root.iterfind('country'):
        idc = element.get('car_code')
        if idc in Countries_result_id:
            Countries_result.append(element.find('name').text)
    
    return Property_result, Countries_result

###a) Longest River

In [267]:
longest_river, countries_river = wrap_question4(root, 'river', 'length')

In [268]:
print 'The longest river is ' + longest_river + ' that is located in ' + str(countries_river)

The longest river is Amazonasthat is located in ['Colombia', 'Brazil', 'Peru']


###b) Largest Lake

In [269]:
largest_lake, countries_lake = wrap_question4(root, 'lake', 'area')

In [270]:
print 'The largest lake is ' + largest_lake + ' that is located in ' + str(countries_lake)

The largest lake is Caspian Sea that is located in ['Russia', 'Iran', 'Turkmenistan', 'Azerbaijan', 'Kazakhstan']


###c) Airport at highest elevation

In [271]:
largest_airport, countries_airport = wrap_question4(root, 'airport', 'elevation')

In [272]:
print 'The airport at highest elevation is ' + largest_airport + ' that is located in ' + str(countries_airport)

The airport at highest elevation is El Alto Intl that is located in ['Bolivia']
